In [ ]:
#%pip install lxml
import os
import s3fs
import pandas as pd
import xml.etree.ElementTree as ET
#from lxml import etree

fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])
with fs.open("s3://lab/pid_name.csv") as f:
   pid = pd.read_csv(f)
with fs.open("s3://lab/pid_orcid.csv") as f:
    orcid = pd.read_csv(f)
"""
with fs.open("s3://lab/conf_net.parquet") as f:
    conf = pd.read_parquet(f)
with fs.open("s3://lab/art_net.parquet") as f:
    art = pd.read_parquet(f)"""




'\nwith fs.open("s3://lab/conf_net.parquet") as f:\n    conf = pd.read_parquet(f)\nwith fs.open("s3://lab/art_net.parquet") as f:\n    art = pd.read_parquet(f)'

In [ ]:
import requests

# 1️⃣ Your ORCID API credentials
CLIENT_ID = ""
CLIENT_SECRET = ""

# 2️⃣ Get an access token (client credentials flow)
def get_access_token(client_id, client_secret):
    url = "https://orcid.org/oauth/token"
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
        "scope": "/read-public"
    }
    resp = requests.post(url, data=data)
    resp.raise_for_status()
    token = resp.json()["access_token"]
    return token

# 3️⃣ Fetch ORCID public record
def fetch_orcid(orcid_id, access_token):
    url = f"https://pub.orcid.org/v3.0/{orcid_id}"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    data = resp.json()

    # Extract name
    person = data.get("person", {})
    given = person.get("name", {}).get("given-names", {}).get("value", "")
    family = person.get("name", {}).get("family-name", {}).get("value", "")
    full_name = f"{given} {family}".strip()

    # Extract affiliations (employments)
    affiliations = []
    for emp in data.get("activities-summary", {}).get("employments", {}).get("employment-summary", []):
        org = emp.get("organization", {})
        org_name = org.get("name")
        if org_name:
            affiliations.append(org_name)

    return {"orcid": orcid_id, "name": full_name, "affiliations": affiliations}

# Example usage
if __name__ == "__main__":
    access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)
    
    orcids = [
        "0000-0002-1825-0097",  # example ORCID
        "0000-0002-2058-0693"
    ]
    
    for orcid_id in orcids:
        info = fetch_orcid(orcid_id, access_token)
        print(info)

{'orcid': '0000-0002-1825-0097', 'name': 'Josiah Carberry', 'affiliations': []}
{'orcid': '0000-0002-2058-0693', 'name': 'Eric Moulines', 'affiliations': []}


In [ ]:
import requests
import json

# 1️⃣ ORCID API credentials
CLIENT_ID = ""
CLIENT_SECRET = ""

# 2️⃣ Get an access token using client credentials flow
def get_access_token(client_id, client_secret):
    url = "https://orcid.org/oauth/token"
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
        "scope": "/read-public"
    }
    resp = requests.post(url, data=data)
    resp.raise_for_status()
    return resp.json()["access_token"]

# 3️⃣ Fetch full ORCID public record
def fetch_full_orcid(orcid_id, access_token):
    url = f"https://pub.orcid.org/v3.0/{orcid_id}"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Accept": "application/json"
    }
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()  # return full JSON

# Example usage
if __name__ == "__main__":
    access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)
    
    orcids = [
        "0000-0002-1825-0097",  # example ORCID
        "0000-0002-2058-0693"
    ]
    
    for orcid_id in orcids:
        full_record = fetch_full_orcid(orcid_id, access_token)
        print(json.dumps(full_record, indent=2))  # pretty-print JSON

{
  "orcid-identifier": {
    "uri": "https://orcid.org/0000-0002-1825-0097",
    "path": "0000-0002-1825-0097",
    "host": "orcid.org"
  },
  "preferences": {
    "locale": "en"
  },
  "history": {
    "creation-method": "WEBSITE",
    "completion-date": {
      "value": 1352727077585
    },
    "submission-date": {
      "value": 1352723476658
    },
    "last-modified-date": {
      "value": 1770681618401
    },
    "claimed": true,
    "source": null,
    "deactivation-date": null,
    "verified-email": true,
    "verified-primary-email": true
  },
  "person": {
    "last-modified-date": {
      "value": 1494016313820
    },
    "name": {
      "created-date": {
        "value": 1460757617078
      },
      "last-modified-date": {
        "value": 1504850007188
      },
      "given-names": {
        "value": "Josiah"
      },
      "family-name": {
        "value": "Carberry"
      },
      "credit-name": null,
      "source": null,
      "visibility": "public",
      "path": "00

In [2]:
name_conf = pd.DataFrame(
    [a for authors in conf["authors"] for a in authors]

)

name_art = pd.DataFrame(
[a for authors in art["authors"] for a in authors]
)

In [3]:
merge_art = pid.merge(name_art, on="name",how="left")
merge_conf = pid.merge(name_conf, on="name",how="left")

In [5]:
df = pd.concat([merge_art,merge_conf]).drop_duplicates()

In [7]:
df = df.dropna()
df

,pid,name,orcid
10,https://dblp.org/pid/69/220,Tien-Tsin Wong,0000-0002-7792-9307
161,https://dblp.org/pid/99/2633,Sylvia C. Pont,0000-0002-9834-9600
177,https://dblp.org/pid/37/6152,Daniel C. Alexander,0000-0003-2439-350X
349,https://dblp.org/pid/01/1485,Kazuhiro Fukui,0000-0002-4201-1096
583,https://dblp.org/pid/73/5043,Zhengyou Zhang,0000-0002-6606-2525
...,...,...,...
12495574,https://dblp.org/pid/17/8263,Saroosh Shabbir,0000-0003-3123-0389
12501002,https://dblp.org/pid/318/3287,Clodomir Silva Lima Neto,0000-0001-9835-9481
12522511,https://dblp.org/pid/414/7334,Gautham Das,0000-0001-5351-9533
12523452,https://dblp.org/pid/135/6070,Diana Catalina Ardila,0000-0001-6553-9216


In [8]:
with fs.open("s3://lab/pid_orcid.csv", "wb") as f:
    df.to_csv(f, index=False)

In [4]:
import requests
import time
from bs4 import BeautifulSoup


headers = {
    "User-Agent": "Mozilla/5.0"
}

results = []

# ----- Loop through links -----
for url in orcid["pid"][:10_000]:
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "html.parser")

        affiliations = []

        # Extract affiliation names
        for block in soup.select('li[itemprop="affiliation"]'):
            name = block.select_one('[itemprop="name"]')
            
            if name:
                affiliations.append(name.get_text(strip=True))

        # Store results
        results.append({
            "profile": url,
            "affiliations": affiliations
        })
        print(len(results))

    except Exception as e:
        print(f"Error scraping {url}: {e}")

    # ----- Cooldown (important!) -----
    time.sleep(0.5)   # change delay if needed


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
Error scraping https://dblp.org/pid/41/2904: 429 Client Error: Too Many Requests for url: https://dblp.org/pid/41/2904.html
Error scraping https://dblp.org/pid/94/104: 429 Client Error: Too Many Requests for url: https://dblp.org/pid/94/104
Error scraping https://dblp.org/pid/01/958: 429 Client Error: Too Many Requests for url: https://dblp.org/pid/01/958
Error scraping https://dblp.org/pid/32/6819: 429 Client Error: Too Many Requests for url: https://dblp.org/pid/32/6819
Error scraping https://dblp.org/pid/19/5997: 429 Client Error: Too Many Requests for url: https://dblp.org/pid/19/5997
Error scraping https://dblp.org/pid/73/1498: 429 Client Error: Too Many Requests for url: https://dblp.org/pid/73/1498
Error scraping https://dblp.org/pid/02/3841: 429 Client Error: Too Many Requests for url: https://dblp.org/pid/02/3841
Error scraping https://dblp.org/pid/82/5175: 429 Client Error: Too Many Requests for url: https:/

KeyboardInterrupt: 

In [ ]:
import asyncio
import aiohttp
import nest_asyncio
from bs4 import BeautifulSoup
import pandas as pd
import random

nest_asyncio.apply()  # Needed for Jupyter

HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

CONCURRENCY = 8   # Safe for DBLP
semaphore = asyncio.Semaphore(CONCURRENCY)

results = []

async def scrape(session, url, retries=3):

    async with semaphore:
        for attempt in range(retries):

            try:
                async with session.get(url, headers=HEADERS, timeout=15) as response:
                    html = await response.text()

                    soup = BeautifulSoup(html, "html.parser")

                    affiliations = [
                        name.get_text(strip=True)
                        for name in soup.select('li[itemprop="affiliation"] [itemprop="name"]')
                    ]

                    results.append({
                        "profile": url,
                        "affiliations": affiliations
                    })

                    return  # success → exit retry loop

            except Exception as e:

                if attempt < retries - 1:
                    wait = 2 ** attempt   # exponential backoff
                    print(f"Retry {attempt+1} for {url} in {wait}s")
                    await asyncio.sleep(random.uniform(0.05, 0.2))
                else:
                    print("Failed:", url, e)



async def main(urls):

    connector = aiohttp.TCPConnector(
    limit=CONCURRENCY,
    ssl=False,              # often fixes SSL resets
    force_close=True        # avoids stale keep-alive sockets
    )

    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [scrape(session, url) for url in urls]
        await asyncio.gather(*tasks)


await main(orcid["pid"][:100_000])

df = pd.DataFrame(results)
df.head()


Retry 1 for https://dblp.org/pid/h/RIHartley in 1s
Retry 2 for https://dblp.org/pid/h/RIHartley in 2s
Retry 1 for https://dblp.org/pid/41/2904 in 1s
Retry 1 for https://dblp.org/pid/01/958 in 1s
Retry 1 for https://dblp.org/pid/94/104 in 1s
Retry 1 for https://dblp.org/pid/32/6819 in 1s
Retry 2 for https://dblp.org/pid/94/104 in 2s
Retry 2 for https://dblp.org/pid/32/6819 in 2s
Retry 1 for https://dblp.org/pid/02/3841 in 1s
Retry 2 for https://dblp.org/pid/01/958 in 2s
Retry 1 for https://dblp.org/pid/82/5175 in 1s
Retry 2 for https://dblp.org/pid/41/2904 in 2s
Failed: https://dblp.org/pid/01/958 Cannot connect to host dblp.org:443 ssl:default [Connection reset by peer]
Retry 2 for https://dblp.org/pid/02/3841 in 2s
Failed: https://dblp.org/pid/94/104 Cannot connect to host dblp.org:443 ssl:default [Connection reset by peer]
Failed: https://dblp.org/pid/32/6819 Cannot connect to host dblp.org:443 ssl:default [Connection reset by peer]
Failed: https://dblp.org/pid/41/2904 Cannot connect

In [ ]:
df = pd.concat(
    [art.explode("authors")["authors"],
     conf.explode("authors")["authors"]],
    ignore_index=True
)

df = df.apply(pd.Series)[['name', 'orcid']]
df

In [ ]:
browser = await p.chromium.launch(
    headless=True,
    args=["--no-sandbox", "--disable-setuid-sandbox"]
)


TargetClosedError: BrowserType.launch: Target page, context or browser has been closed
Browser logs:

<launching> /home/onyxia/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/chrome-headless-shell --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side-phishing-detection --disable-component-extensions-with-background-pages --disable-component-update --no-default-browser-check --disable-default-apps --disable-dev-shm-usage --disable-extensions --disable-features=AvoidUnnecessaryBeforeUnloadCheckSync,BoundaryEventDispatchTracksNodeRemoval,DestroyProfileOnBrowserClose,DialMediaRouteProvider,GlobalMediaControls,HttpsUpgrades,LensOverlay,MediaRouter,PaintHolding,ThirdPartyStoragePartitioning,Translate,AutoDeElevate,RenderDocument,OptimizationHints --enable-features=CDPScreenshotNewSurface --allow-pre-commit-

In [16]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

URL = "https://orcid.org/0000-0002-2058-0693"

async def scrape_orcid_panels():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=["--no-sandbox"])
        page = await browser.new_page()
        await page.goto(URL, timeout=60000)

        # Scroll to trigger lazy loading
        for _ in range(5):
            await page.evaluate("window.scrollBy(0, document.body.scrollHeight)")
            await page.wait_for_timeout(1000)

        await page.wait_for_selector("#activities")

        panels = await page.query_selector_all("orcid-panel")
        data = []
        for idx, panel in enumerate(panels):
            text = await panel.inner_text()
            if not text.strip():
                continue
            data.append({
                "panel_index": idx,
                "text": text
            })

        await browser.close()
        return data

# In Jupyter
df = pd.DataFrame(await scrape_orcid_panels())
df.head()


TargetClosedError: BrowserType.launch: Target page, context or browser has been closed
Browser logs:

<launching> /home/onyxia/.cache/ms-playwright/chromium_headless_shell-1208/chrome-headless-shell-linux64/chrome-headless-shell --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side-phishing-detection --disable-component-extensions-with-background-pages --disable-component-update --no-default-browser-check --disable-default-apps --disable-dev-shm-usage --disable-extensions --disable-features=AvoidUnnecessaryBeforeUnloadCheckSync,BoundaryEventDispatchTracksNodeRemoval,DestroyProfileOnBrowserClose,DialMediaRouteProvider,GlobalMediaControls,HttpsUpgrades,LensOverlay,MediaRouter,PaintHolding,ThirdPartyStoragePartitioning,Translate,AutoDeElevate,RenderDocument,OptimizationHints --enable-features=CDPScreenshotNewSurface --allow-pre-commit-

In [ ]:
with fs.open(f"s3://lab/orcids.parquet", "wb") as f:
            df.to_parquet(f, index=False)

,title,journal,year,authors,dblp_uri,doi
0,Auswirkung der Digitalisierung auf die Systeml...,Elektrotech. Informationstechnik,2019,"{'name': 'Kai Schlabitz', 'orcid': None}",journals/ei/Schlabitz19,https://doi.org/10.1007/s00502-019-0687-y
1,EMF-Personenschutz: Neue Aspekte in der numeri...,Elektrotech. Informationstechnik,2020,"{'name': 'Richard berbacher', 'orcid': None}",journals/ei/UberbacherC20,https://doi.org/10.1007/s00502-020-00791-z
1,EMF-Personenschutz: Neue Aspekte in der numeri...,Elektrotech. Informationstechnik,2020,"{'name': 'Stefan Cecil', 'orcid': None}",journals/ei/UberbacherC20,https://doi.org/10.1007/s00502-020-00791-z
2,Zur Genesis der Forschungsstelle fr Integriert...,Elektrotech. Informationstechnik,2022,"{'name': 'Herbert Mang', 'orcid': None}",journals/ei/Mang22,https://doi.org/10.1007/s00502-022-01049-6
3,100 % erneuerbare Energie fr sterreichs Indust...,Elektrotech. Informationstechnik,2021,"{'name': 'Sophie Knttner', 'orcid': None}",journals/ei/KnottnerGDD21,https://doi.org/10.1007/s00502-021-00953-7
...,...,...,...,...,...,...
3992220,Analysis of projected hydrological behavior of...,Hydrology and Earth System Sciences,2012,"{'name': 'Rita Ley', 'orcid': None}",persons/CasperGGGHLR12,https://doi.org/10.5194/hess-16-409-2012
3992220,Analysis of projected hydrological behavior of...,Hydrology and Earth System Sciences,2012,"{'name': 'Andreas Rock', 'orcid': None}",persons/CasperGGGHLR12,https://doi.org/10.5194/hess-16-409-2012
3992221,Common Subexpression Identification in General...,"Technical Rep. UKSC 0060, IBM United Kingdom S...",1974,"{'name': 'Patrick A. V. Hall', 'orcid': None}",persons/Hall74,None
3992222,Interactive Support for Non-Programmers: The R...,"Research Report / RJ / IBM / San Jose, California",1974,"{'name': 'E. F. Codd', 'orcid': None}",persons/CoddD74,None
